# **Short-Text Topic Modeling**

In [ ]:
# !pip install bertopic
!pip install datasets
!pip install gensim==4.1.2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 346 kB 9.5 MB/s 
     |████████████████████████████████| 140 kB 69.1 MB/s 
     |████████████████████████████████| 86 kB 6.7 MB/s 
     |████████████████████████████████| 86 kB 7.5 MB/s 
     |████████████████████████████████| 1.1 MB 62.1 MB/s 
     |████████████████████████████████| 212 kB 72.8 MB/s 
     |████████████████████████████████| 596 kB 65.7 MB/s 
     |████████████████████████████████| 127 kB 73.5 MB/s 
     |████████████████████████████████| 94 kB 3.2 MB/s 
     |████████████████████████████████| 271 kB 55.7 MB/s 
     |████████████████████████████████| 144 kB 61.3 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
     

In [ ]:
from IPython.display import clear_output
import gzip
import shutil
# import os
# import linecache
# from shutil import copyfile
# import ipywidgets as widgets

import pandas as pd
import numpy as np
import gensim
# import pythainlp
from datasets import load_dataset
# import tweepy

## **Data Acquisition**

load data in to the workspace
- AG News 
- COVID-19 Twitter

In [ ]:
# !pip install twarc 
# !pip install tweepy==3.8.0
# !pip install argparse 
# !pip install xtract 
# !pip install wget
# clear_output()

### **COVID-19 Twitter Query**

In [ ]:
twitter = pd.read_csv("clean-dataset-filtered.tsv", sep="\t")

FileNotFoundError: ignored

In [ ]:
list(twitter.tweet_id)

In [ ]:
dataset_URL = "https://github.com/thepanacealab/covid19_twitter/blob/master/dailies/2022-03-19/2022-03-19-dataset.tsv.gz?raw=true"

#Downloads the dataset (compressed in a GZ format)
#!wget dataset_URL -O clean-dataset.tsv.gz
wget.download(dataset_URL, out='clean-dataset.tsv.gz')

#Unzips the dataset and gets the TSV dataset
with gzip.open('clean-dataset.tsv.gz', 'rb') as f_in:
    with open('clean-dataset.tsv', 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)

#Deletes the compressed GZ file
os.unlink("clean-dataset.tsv.gz")

In [ ]:
df = pd.read_csv('clean-dataset.tsv', sep = "\t")
lang_list = df.lang.unique()
lang_list= sorted(np.append(lang_list,'all'))
lang_picker = widgets.Dropdown(options=lang_list, value="all")
lang_picker

Dropdown(options=('all', 'am', 'ar', 'bg', 'bn', 'ca', 'ckb', 'cs', 'cy', 'da', 'de', 'dv', 'el', 'en', 'es', …

In [ ]:
#Creates a new clean dataset with the specified language (if specified)
filtered_language = lang_picker.value

#If no language specified, it will get all records from the dataset
if filtered_language == "":
  copyfile('clean-dataset.tsv', 'clean-dataset-filtered.tsv')

#If language specified, it will create another tsv file with the filtered records
else:
  filtered_tw = list()
  current_line = 1
  with open("clean-dataset.tsv") as tsvfile:
    tsvreader = csv.reader(tsvfile, delimiter="\t")

    if current_line == 1:
      filtered_tw.append(linecache.getline("clean-dataset.tsv", current_line))

      for line in tsvreader:
        if line[3] == filtered_language:
          filtered_tw.append(linecache.getline("clean-dataset.tsv", current_line))
        current_line += 1

  print('\033[1mShowing first 5 tweets from the filtered dataset\033[0m')
  print(filtered_tw[1:(6 if len(filtered_tw) > 6 else len(filtered_tw))])

  with open('clean-dataset-filtered.tsv', 'w') as f_output:
      for item in filtered_tw:
          f_output.write(item)

Showing first 5 tweets from the filtered dataset
['1505031523711344643\t2022-03-19\t04:00:52\ten\tNULL\n', '1505031527251394561\t2022-03-19\t04:00:52\ten\tNULL\n', '1505031528551948293\t2022-03-19\t04:00:53\ten\tNULL\n', '1505031529117982724\t2022-03-19\t04:00:53\ten\tNULL\n', '1505031530783121408\t2022-03-19\t04:00:53\ten\tNULL\n']


In [ ]:
import json
import tweepy
from tweepy import OAuthHandler

# Authenticate
CONSUMER_KEY = "XOMdfQltmfYfDuLMgFaXWkuSy" #@param {type:"string"}
CONSUMER_SECRET_KEY = "oGeCwmitKZdOKgFqdjE36bmkFEpvJEmriozDuZ5vhu7DF0Epzy" #@param {type:"string"}
ACCESS_TOKEN_KEY = "2615165239-Co7C80rwZPWlKB5chrDuQhdON64q7xqJxTyQx5V" #@param {type:"string"}
ACCESS_TOKEN_SECRET_KEY = "VJoEbNxEJ88FhOLWBOiU7bKrir4j2IWfWtTOB0kTctSi2" #@param {type:"string"}

#Creates a JSON Files with the API credentials
with open('api_keys.json', 'w') as outfile:
    json.dump({
    "consumer_key":CONSUMER_KEY,
    "consumer_secret":CONSUMER_SECRET_KEY,
    "access_token":ACCESS_TOKEN_KEY,
    "access_token_secret": ACCESS_TOKEN_SECRET_KEY
}, outfile)

In [ ]:
# !python -m wget https://raw.githubusercontent.com/thepanacealab/SMMT/master/data_acquisition/get_metadata.py -o get_metadata.py


In [ ]:
auth = tweepy.AppAuthHandler(CONSUMER_KEY, CONSUMER_SECRET_KEY)

api = tweepy.API(auth, wait_on_rate_limit=True)

if (not api):
   print ("Can't Authenticate")
   sys.exit(-1)

In [ ]:
!python get_metadata.py -i clean-dataset-filtered.tsv -o hydrated_tweets -k api_keys.json

Your twitter api credentials are valid.
hydrated_tweets
tab seperated file, using \t delimiter

Traceback (most recent call last):
  File "get_metadata.py", line 187, in <module>
    main()
  File "get_metadata.py", line 183, in main
    f.writerow([data["favorite_count"], data["source"], data["text"].encode('utf-8'), data["in_reply_to_screen_name"], data["is_retweet"], data["created_at"], data["retweet_count"], data["id_str"].encode('utf-8')])
  File "C:\Users\pond\anaconda3\lib\encodings\cp1252.py", line 19, in encode
    return codecs.charmap_encode(input,self.errors,encoding_table)[0]
UnicodeEncodeError: 'charmap' codec can't encode character '\u039f' in position 16: character maps to <undefined>



total ids: 134851
metadata collection complete
creating master json file
currently getting 0 - 100
currently getting 100 - 200
currently getting 200 - 300
currently getting 300 - 400
currently getting 400 - 500
currently getting 500 - 600
currently getting 600 - 700
currently getting 700 - 800
currently getting 800 - 900
currently getting 900 - 1000
currently getting 1000 - 1100
currently getting 1100 - 1200
currently getting 1200 - 1300
currently getting 1300 - 1400
currently getting 1400 - 1500
currently getting 1500 - 1600
currently getting 1600 - 1700
currently getting 1700 - 1800
currently getting 1800 - 1900
currently getting 1900 - 2000
currently getting 2000 - 2100
currently getting 2100 - 2200
currently getting 2200 - 2300
currently getting 2300 - 2400
currently getting 2400 - 2500
currently getting 2500 - 2600
currently getting 2600 - 2700
currently getting 2700 - 2800
currently getting 2800 - 2900
currently getting 2900 - 3000
currently getting 3000 - 3100
currently getting

### **AG News**

In [ ]:
ag_news = load_dataset("ag_news")
clear_output()

In [ ]:
train_ag = ag_news['train']['text']
test_ag = ag_news['test']['text']

In [ ]:
pd.DataFrame(train_ag, columns = ['text'])

,text
0,Wall St. Bears Claw Back Into the Black (Reute...
1,Carlyle Looks Toward Commercial Aerospace (Reu...
2,Oil and Economy Cloud Stocks' Outlook (Reuters...
3,Iraq Halts Oil Exports from Main Southern Pipe...
4,"Oil prices soar to all-time record, posing new..."
...,...
119995,Pakistan's Musharraf Says Won't Quit as Army C...
119996,Renteria signing a top-shelf deal Red Sox gene...
119997,Saban not going to Dolphins yet The Miami Dolp...
119998,Today's NFL games PITTSBURGH at NY GIANTS Time...


### **Covid-19 Twitter**

In [ ]:
!gdown --id 1aXkVexGaw5Ei6045XViUwbJQLJWnYVA7

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1aXkVexGaw5Ei6045XViUwbJQLJWnYVA7
To: /content/hydrated_tweets_short.json
100% 44.2M/44.2M [00:00<00:00, 278MB/s]


In [ ]:
tweets = pd.read_json("hydrated_tweets_short.json", lines=True)

In [ ]:
tweets_text = tweets["text"]

In [ ]:
tweets_text

0         RT @CDCgov: New @CDCMMWR study finds #COVID19 ...
1         RT @BeingCharisBlog: CDC: "A person with any o...
2         RT @USMortality: WOW look at Vietnam! Basicall...
3         RT @AppSame: 997,136#Covid19 deaths Biden is a...
4         RT @AppSame: 997,136#Covid19 deaths Biden is a...
                                ...                        
123595    RT @naearthiive: Announcement from #1BLOODFOR2...
123596    Still one-way masking: inside retail spaces an...
123597    RT @MarkusMannheim: About 15.4% of Canberrans ...
123598    RT @terminalcwo: We should all rally against t...
123599    RT @uche_blackstock: Now would be a great time...
Name: text, Length: 123600, dtype: object

# **Data Preparation**

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
import re
from nltk.tokenize import word_tokenize, RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import LancasterStemmer
from nltk.stem import WordNetLemmatizer
from gensim.parsing.preprocessing import strip_short
from gensim.models import Phrases

In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [ ]:
stop_words = set(stopwords.words('english'))

In [ ]:
tokenizer = RegexpTokenizer(r'\w+')

In [ ]:
lemmatizer = WordNetLemmatizer()

In [ ]:
def remove_special(sentence, output_option = "list"):
    # convert all of documents to lowercases
    cleantext = sentence.lower()
    
    # tokenize text with regexp tokenizer which will eliminates special characters
    cleantext = tokenizer.tokenize(cleantext) 
    
    # filtering out the stop words
    cleantext = [w for w in cleantext if not w.lower() in stop_words] 
    
    # remove number but not word that contains number
    cleantext = [w for w in cleantext if not w.isnumeric()]
    
    # filtering out the word which have length less than 1
    cleantext = [w for w in cleantext if len(w) > 2]
    # Stemmer or Lemmer
#     stem_words=[stemmer.stem(w) for w in filtered_words]
    lemma_words=[lemmatizer.lemmatize(w) for w in cleantext]
    if output_option == "list":
        return cleantext
    elif output_option == "text":
        return " ".join(cleantext)
    # return cleantext

In [ ]:
remove_special(train_ag[10], "text")

'oil economy cloud stocks outlook new york reuters soaring crude prices plus worries economy outlook earnings expected hang stock market next week depth summer doldrums'

In [ ]:
remove_special(tweets["text"][0], "text")

'cdcgov new cdcmmwr study finds covid19 vaccines continued highly effective protecting adults put ventilator'

In [ ]:
preprocess_tweets = list(map(remove_special, list(tweets["text"])))

In [ ]:
preprocess_tweets_text = [remove_special(i, "text") for i in list(tweets["text"])]

In [ ]:
preprocess_ag = list(map(remove_special, train_ag))

In [ ]:
preprocess_ag_test = list(map(remove_special, test_ag))

In [ ]:
preprocess_ag_text = [remove_special(i, "text") for i in train_ag]

In [ ]:
preprocess_ag[0]

['wall',
 'bears',
 'claw',
 'back',
 'black',
 'reuters',
 'reuters',
 'short',
 'sellers',
 'wall',
 'street',
 'dwindling',
 'band',
 'ultra',
 'cynics',
 'seeing',
 'green']

In [ ]:
train_ag[0]

"Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\\band of ultra-cynics, are seeing green again."

Add Bi-gram to AG News Documents

In [ ]:
# Add bigrams and trigrams to docs (only ones that appear 20 times or more).
bigram = Phrases(preprocess_ag, min_count=20)
for idx in range(len(preprocess_ag)):
    for token in bigram[preprocess_ag[idx]]:
        if '_' in token:
            # Token is a bigram, add to document.
            preprocess_ag[idx].append(token)

In [ ]:
bigram = Phrases(preprocess_tweets, min_count=20)
for idx in range(len(preprocess_tweets)):
    for token in bigram[preprocess_tweets[idx]]:
        if '_' in token:
            # Token is a bigram, add to document.
            preprocess_tweets[idx].append(token)

## **Bag-of-Word (Gensim)**

In [ ]:
from gensim.corpora import Dictionary

In [ ]:
dictionary_ag = Dictionary(preprocess_ag)
dictionary_tweets = Dictionary(preprocess_tweets)

# # Filter out words that occur less than 20 documents, or more than 50% of the documents.
# dictionary_ag.filter_extremes(no_below=20, no_above=0.5)
# dictionary_tweets.filter_extremes(no_below=20, no_above=0.5)

clear_output()

In [ ]:
print(dictionary_tweets)

Dictionary(88212 unique tokens: ['adults', 'cdcgov', 'cdcmmwr', 'continued', 'covid19']...)


In [ ]:
print(dictionary_ag)

Dictionary(67611 unique tokens: ['back', 'band', 'bears', 'black', 'claw']...)


In [ ]:
# create bag of word vectors with doc2bow function
corpus_ag = [dictionary_ag.doc2bow(doc) for doc in preprocess_ag] # convert each documents in preprocessed_ag to bag of word
corpus_tweets = [dictionary_tweets.doc2bow(doc) for doc in preprocess_tweets]

In [ ]:
corpus_ag_test = [dictionary_ag.doc2bow(doc) for doc in preprocess_ag_test]

In [ ]:
# sample of bag of word representation of document
# for i in corpus_ag[:1]:
#     for index in i:  
#         # print(dictionary[0])
#         print(dictionary_ag[index[0]], index[1])
for i in corpus_tweets[:1]:
    for index in i:
        # print(dictionary[0])
        print(dictionary_tweets[index[0]], index[1])

adults 1
cdcgov 1
cdcmmwr 1
continued 1
covid19 1
effective 1
finds 1
highly 1
highly_effective 1
new 1
new_cdcmmwr 1
protecting 1
protecting_adults 1
put 1
put_ventilator 1
study 1
study_finds 1
vaccines 1
vaccines_continued 1
ventilator 1


## **CountVectorizer sklearn**

In [ ]:
vectorizer = CountVectorizer(preprocessor=None)
countvec_ag = vectorizer.fit_transform(preprocess_ag_text)
# vectorizer.get_feature_names_out()

## **WORD2VEC**

In [ ]:
from gensim.test.utils import common_texts
from gensim.models import Word2Vec

adding document #0 to Dictionary(0 unique tokens: [])
built Dictionary(12 unique tokens: ['computer', 'human', 'interface', 'response', 'survey']...) from 9 documents (total 29 corpus positions)


In [ ]:
word2vec_model = Word2Vec(sentences=preprocess_ag, vector_size=100, window=5, min_count=1, workers=4)

collecting all words and their counts
PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
PROGRESS: at sentence #10000, processed 293325 words, keeping 24839 word types
PROGRESS: at sentence #20000, processed 582158 words, keeping 33881 word types
PROGRESS: at sentence #30000, processed 871275 words, keeping 39866 word types
PROGRESS: at sentence #40000, processed 1160882 words, keeping 44461 word types
PROGRESS: at sentence #50000, processed 1449252 words, keeping 48395 word types
PROGRESS: at sentence #60000, processed 1738367 words, keeping 52020 word types
PROGRESS: at sentence #70000, processed 2031285 words, keeping 55198 word types
PROGRESS: at sentence #80000, processed 2318569 words, keeping 58136 word types
PROGRESS: at sentence #90000, processed 2601523 words, keeping 61057 word types
PROGRESS: at sentence #100000, processed 2884831 words, keeping 63719 word types
PROGRESS: at sentence #110000, processed 3166671 words, keeping 66159 word types
collected 68417 w

In [ ]:
word2vec_model.train([["hello", "world"]], total_examples=1, epochs=1)

Effective 'alpha' higher than previous training cycles
Word2Vec lifecycle event {'msg': 'training model with 4 workers on 68417 vocabulary and 100 features, using sg=0 hs=0 sample=0.001 negative=5 window=5 shrink_windows=True', 'datetime': '2022-04-25T23:24:49.276404', 'gensim': '4.1.2', 'python': '3.8.12 (default, Oct 12 2021, 03:01:40) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.22598-SP0', 'event': 'train'}
worker thread finished; awaiting finish of 3 more threads
worker thread finished; awaiting finish of 2 more threads
worker thread finished; awaiting finish of 1 more threads
worker thread finished; awaiting finish of 0 more threads
EPOCH - 1 : training on 2 raw words (2 effective words) took 0.0s, 623 effective words/s
Word2Vec lifecycle event {'msg': 'training on 2 raw words (2 effective words) took 0.0s, 267 effective words/s', 'datetime': '2022-04-25T23:24:49.284471', 'gensim': '4.1.2', 'python': '3.8.12 (default, Oct 12 2021, 03:01:40) [MSC v.1916 64 bit (AMD64

(2, 2)

## **Uncategorized Code**

In [ ]:
from pythainlp.word_vector import WordVector
from gensim.models import Word2Vec

loading projection weights from C:\Users\pond\pythainlp-data\thai2vec.bin
KeyedVectors lifecycle event {'msg': 'loaded (51358, 300) matrix of type float32 from C:\\Users\\pond\\pythainlp-data\\thai2vec.bin', 'binary': True, 'encoding': 'utf8', 'datetime': '2022-04-25T23:02:21.546245', 'gensim': '4.1.2', 'python': '3.8.12 (default, Oct 12 2021, 03:01:40) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.22598-SP0', 'event': 'load_word2vec_format'}


In [ ]:
wv = WordVector()

loading projection weights from C:\Users\pond\pythainlp-data\thai2vec.bin
KeyedVectors lifecycle event {'msg': 'loaded (51358, 300) matrix of type float32 from C:\\Users\\pond\\pythainlp-data\\thai2vec.bin', 'binary': True, 'encoding': 'utf8', 'datetime': '2022-03-25T19:16:33.251466', 'gensim': '4.1.2', 'python': '3.8.12 (default, Oct 12 2021, 03:01:40) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.22543-SP0', 'event': 'load_word2vec_format'}


In [ ]:
sentence = 'ฉันรักประเทศไทย'
sentence_vector = wv.sentence_vectorizer(sentence)
print(sentence_vector)

[[ 1.71394671e-01 -2.43284330e-01 -1.41679967e-02  2.96576001e-01
  -2.22379004e-01 -7.44023304e-02 -5.33566376e-03  9.41766674e-03
  -1.74809662e-01 -1.44934667e-01 -3.31633329e-01 -1.30679997e-01
   1.81364005e-01  1.67169669e-01 -2.02283661e-01 -1.57379980e-02
  -7.14666670e-02  2.12028998e-01  4.89439977e-02 -2.99740005e-02
  -1.26108664e-01  2.74677332e-01  9.74936659e-02  6.31487002e-01
  -3.15139999e-01  4.49893996e-01  1.27577665e-01 -1.58133171e-03
  -2.14869662e-01 -5.12753278e-02 -1.92380051e-02 -2.24013329e-01
   2.40447673e-01 -2.18494669e-01 -1.98223218e-02 -2.72138665e-01
  -1.06574662e-01  1.65506682e-02  1.22765000e-01  8.32156638e-02
   1.25510022e-02  1.22485672e-01 -3.44349996e-02 -6.50096685e-02
  -3.66518664e-01 -3.11753343e-01  3.20426704e-02  2.93954653e-01
   3.07067662e-01  3.33423336e-01  1.35850003e-02  8.08779954e-02
   1.23463670e-01 -1.24875989e-01 -1.04736676e-01 -5.93169990e-02
   5.11276013e-01 -3.85653277e-02  1.80944003e-01 -1.93367337e-01
   9.31639

In [ ]:
porter = PorterStemmer()
print(porter.stem("connection"))

connect


In [ ]:
from nltk.tokenize import word_tokenize

In [ ]:
sentence = "I love Thailand"
print(word_tokenize(sentence))

['I', 'love', 'Thailand']


In [ ]:
import nltk

In [ ]:
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [ ]:
import gensim.downloader as api
wv = api.load('word2vec-google-news-300')

Creating C:\Users\pond/gensim-data


[=================================================-] 98.2% 1632.2/1662.8MB downloaded

In [ ]:
import pandas as pd

In [ ]:
test = {
    "This is": [1],
    "is a ": [1],
    "a sentence": [1],
    "This": [1],
    "is": [1],
    "a": [1],
    "sentence": [1]
}

In [ ]:
pd.DataFrame(test, index=["document"])

,This is,is a,a sentence,This,is,a,sentence
document,1,1,1,1,1,1,1


# **Modeling**

In [ ]:
# from bertopic import BERTopic

## **Latent Dirichlet Allocation**

Parameters of Gensim LDA modules:
- **corpus**: set of documents matrix
- **num_topics**: number of latent topics need to extract of the corpus
- **id2word**: dictionary of mapping from IDs to each word from dictionary
- **distributed**: define whether implement distributed computing or not
- **chunksize**: number of documents to be used in each training chunk
- **passes**: Number of passes through the corpus during training
- **update_every**: number of documents to be iterates through for each updates, 0 for batch learning, > 1 for online iterative learning
- **alpha** (float, ndarray of float): prior belief of document-topic distribution. 
    - "symmetric": (default) Uses a fixed symmetric prior of 1.0 / num_topics
    - "asymmetric": Uses a fixed normalized asymmetric prior of 1.0 / (topic_index + sqrt(num_topics))
    - "auto": Learns an asymmetric prior from the corpus (not available if distributed==True).
- **eta** (same as alpha):  prior belief of topic-word distribution
    - "symmetric": (default) Uses a fixed symmetric prior of 1.0 / num_topics
    - "auto": Learns an asymmetric prior from the corpus.
- **decay** (number between (0.5, 1]): use for weighing what percentage of the previous lambda value is forgotten when each new document is examined
- **offset**: Hyper-parameter that controls how much we will slow down the first steps the first few iterations
- **eval_every**: Log perplexity is estimated every that many updates
- **iterations**: Maximum number of iterations through the corpus when inferring the topic distribution of a corpus
- **gammar_threshold**: Minimum change in the value of the gamma parameters to continue iterating
- **minimum_probability**: minimum_probability
- **per_word_topics**: If True, the model also computes a list of topics, sorted in descending order of most likely topics for each word, along with their phi values multiplied by the feature length

In [ ]:
# from gensim.test.utils import common_texts
# from gensim.corpora.dictionary import Dictionary
from gensim.models import LdaModel
from gensim.models.coherencemodel import CoherenceModel

clear_output()

## **AG News Model**

In [ ]:
# num_topics = 5 # number of topic want to extract from the corpus
chunksize = 2000 # number of document which will process at a time, increase chunksize will speed up the model
iterations = 300

# Make a index to word dictionary.
temp = dictionary_ag[0]  # This is only to "load" the dictionary.
id2word_ag = dictionary_ag.id2token

model_ag = LdaModel(
    corpus=corpus_ag, # corpus, the set of documents
    id2word=id2word_ag,
    chunksize=chunksize,
    alpha='auto',
    eta='auto',
    iterations=iterations, # number of iteration
    num_topics=5, # k topics to extract
)
clear_output()
print("LDA trained")

LDA trained


In [ ]:
# model_ag.print_topics(num_topics = -1, num_words = 12)
model_ag.top_topics(corpus_ag, topn=10)

[([(0.011624777, 'new'),
   (0.0104185715, 'company'),
   (0.00939975, 'inc'),
   (0.008058279, 'reuters'),
   (0.007825126, 'microsoft'),
   (0.0076052253, 'corp'),
   (0.007365865, 'com'),
   (0.007124275, 'software'),
   (0.006454072, 'said'),
   (0.0057605603, 'fullquote')],
  -2.350291316015687),
 ([(0.014512983, 'reuters'),
   (0.010513446, 'oil'),
   (0.00909855, 'new'),
   (0.008537114, 'year'),
   (0.0069280635, 'percent'),
   (0.006369867, 'said'),
   (0.0060104546, 'prices'),
   (0.00525806, 'sales'),
   (0.005013871, 'dollar'),
   (0.004944121, 'quarter')],
  -2.705014148599016),
 ([(0.00825931, 'two'),
   (0.0063454364, 'iraq'),
   (0.0058322223, 'first'),
   (0.0054756263, 'game'),
   (0.005457543, 'night'),
   (0.005282717, 'coach'),
   (0.0049549337, 'sunday'),
   (0.004837443, 'season'),
   (0.0047567193, 'three'),
   (0.004714105, 'one')],
  -3.049117358975874),
 ([(0.009320632, 'said'),
   (0.009123361, 'quot'),
   (0.008018611, 'president'),
   (0.005715348, 'afp'),

In [ ]:
model_ag

In [ ]:
preprocess_ag_test[0]

['fears',
 'pension',
 'talks',
 'unions',
 'representing',
 'workers',
 'turner',
 'newall',
 'say',
 'disappointed',
 'talks',
 'stricken',
 'parent',
 'firm',
 'federal',
 'mogul']

In [ ]:
model_ag[corpus_ag_test[0]]

[(0, 0.116204344),
 (1, 0.25825498),
 (2, 0.21311788),
 (3, 0.10538789),
 (4, 0.30703488)]

In [ ]:
test = 0
for i in model_ag[corpus_ag_test[0]]:
  test += i[1]
print(test)

0.9999999850988388


### **K selection**

In [ ]:
coherence_ag = {
    "u_mass": [],
    "c_v": [], 
    "c_uci": [],
    "c_npmi": []
}

In [ ]:
methods = list(coherence_ag.keys())
methods

['u_mass', 'c_v', 'c_uci', 'c_npmi']

In [ ]:
for num_topic in range(2, 10, 1):
    print(f"currently run {num_topic} topic number")
    temp = dictionary_ag[0]
    id2word_ag = dictionary_ag.id2token
    model = LdaModel(
        corpus=corpus_ag, # corpus, the set of documents
        id2word=id2word_ag,
        chunksize=2000,
        alpha='auto',
        eta='auto',
        iterations=300, # number of iteration
        num_topics=num_topic, # k topics to extract
    )
    for method in methods:
        cm = CoherenceModel(model=model, corpus=corpus_ag, dictionary=dictionary_ag, coherence=method, texts=preprocess_ag)
        coherence_ag[method].append(cm.get_coherence())
    clear_output()

In [ ]:
pd.DataFrame(coherence_ag).to_csv("lda_agnews_cm2-9.csv")

In [ ]:
coherence_ag

In [ ]:
ag_results = pd.DataFrame(coherence_ag)

#### **Model with Best K**

In [ ]:
num_topics = 4 # number of topic want to extract from the corpus
chunksize = 2000 # number of document which will process at a time, increase chunksize will speed up the model
iterations = 200

# Make a index to word dictionary.
temp = dictionary_ag[0]  # This is only to "load" the dictionary.
id2word_ag = dictionary_ag.id2token

model_ag = LdaModel(
    corpus=corpus_ag, # corpus, the set of documents
    id2word=id2word_ag,
    chunksize=chunksize,
    alpha='auto',
    eta='auto',
    iterations=iterations, # number of iteration
    num_topics=5, # k topics to extract
)
clear_output()
print("LDA trained")

LDA trained


## **Twitter COVID-19 Model**

In [ ]:
# Set training parameters.
num_topics = 5 # number of topic want to extract from the corpus
chunksize = 2000 # number of document which will process at a time, increase chunksize will speed up the model
# passes = 20
iterations = 400
# eval_every = None  # Don't evaluate model perplexity, takes too much time.

# Make a index to word dictionary.
temp = dictionary_tweets[0]  # This is only to "load" the dictionary.
id2word_tweets = dictionary_tweets.id2token

model_tweet = LdaModel(
    corpus=corpus_tweets, # corpus, the set of documents
    id2word=id2word_tweets,
    chunksize=chunksize,
    alpha='auto',
    eta='auto',
    iterations=iterations, # number of iteration
    num_topics=num_topics, # k topics to extract
)
clear_output()
print("LDA trained")

LDA trained


In [ ]:
# model_ag.print_topics(num_topics = -1)
test = pd.DataFrame()
topics_tweets = model_ag.top_topics(corpus_tweets)
for i in range(len(topics_tweets)):
  temp = []
  for j in range(14):
    # print(i[0][j][1])
    temp.append(topics_tweets[i][0][j][1])
  print(temp)
  test[i] = temp

['new', 'company', 'said', 'inc', 'reuters', 'corp', 'million', 'microsoft', 'software', 'com', 'billion', 'deal', 'fullquote', 'business']
['said', 'reuters', 'president', 'search', 'government', 'people', 'google', 'security', 'palestinian', 'minister', 'officials', 'leader', 'election', 'killed']
['oil', 'reuters', 'new', 'year', 'york', 'new_york', 'prices', 'dollar', 'percent', 'stocks', 'friday', 'wednesday', 'points', 'thursday']
['iraq', 'afp', 'said', 'talks', 'united', 'india', 'world', 'cup', 'union', 'south', 'test', 'two', 'european', 'iraqi']
['game', 'new', 'one', 'quot', 'two', 'first', 'season', 'night', 'last', 'coach', 'time', 'team', 'back', 'year']


In [ ]:
test.to_csv("lda_topics_tweets.csv")

### **K Selection**

In [ ]:
coherence_tweets = {
    "u_mass": [],
    "c_v": [], 
    "c_uci": [],
    "c_npmi": []
}

In [ ]:
for num_topic in range(2, 10, 1):
    print(f"currently run {num_topic} topic number")
    temp = dictionary_tweets[0]  # This is only to "load" the dictionary.
    id2word_tweets = dictionary_tweets.id2token
    model = LdaModel(
        corpus=corpus_tweets, # corpus, the set of documents
        id2word=id2word_tweets,
        chunksize=2000,
        alpha='auto',
        eta='auto',
        iterations=300, # number of iteration
        num_topics=num_topic, # k topics to extract
    )

    for method in methods:
        cm = CoherenceModel(model=model, corpus=corpus_tweets, dictionary=dictionary_tweets, coherence=method , texts=preprocess_tweets)
        coherence_tweets[method].append(cm.get_coherence())
    clear_output()

In [ ]:
coherence_tweets

In [ ]:
pd.DataFrame(coherence_tweets).to_csv("lda_tweets_cm2-9.csv")

In [ ]:
pd.DataFrame(coherence_tweets)

,u_mass,c_v,c_uci,c_npmi
0,-5.120832,0.542765,-3.811937,0.038112
1,-6.561691,0.253730,-3.709768,-0.068408
2,-7.500173,0.302269,-4.342702,-0.111959
3,-8.780821,0.321706,-6.061837,-0.140554
4,-8.906507,0.333035,-5.121459,-0.157372
5,-9.298940,0.418685,-5.910281,-0.151578
6,-8.566949,0.358308,-5.283828,-0.158440
7,-9.898911,0.409522,-6.667513,-0.168053


### **Model with Best K**

In [ ]:
tweets_result = pd.DataFrame(coherence_tweets)

In [ ]:
top_topics = model.top_topics(corpus_ag) #, num_words=20)

# Average topic coherence is the sum of topic coherences of all topics, divided by the number of topics.
avg_topic_coherence = sum([t[1] for t in top_topics]) / num_topics
print('Average topic coherence: %.4f.' % avg_topic_coherence)

from pprint import pprint
pprint(top_topics)

CorpusAccumulator accumulated stats from 1000 documents
CorpusAccumulator accumulated stats from 2000 documents
CorpusAccumulator accumulated stats from 3000 documents
CorpusAccumulator accumulated stats from 4000 documents
CorpusAccumulator accumulated stats from 5000 documents
CorpusAccumulator accumulated stats from 6000 documents
CorpusAccumulator accumulated stats from 7000 documents
CorpusAccumulator accumulated stats from 8000 documents
CorpusAccumulator accumulated stats from 9000 documents
CorpusAccumulator accumulated stats from 10000 documents
CorpusAccumulator accumulated stats from 11000 documents
CorpusAccumulator accumulated stats from 12000 documents
CorpusAccumulator accumulated stats from 13000 documents
CorpusAccumulator accumulated stats from 14000 documents
CorpusAccumulator accumulated stats from 15000 documents
CorpusAccumulator accumulated stats from 16000 documents
CorpusAccumulator accumulated stats from 17000 documents
CorpusAccumulator accumulated stats from

Average topic coherence: -4.1924.
[([(0.01846875, 'reuters'),
   (0.013553397, 'said'),
   (0.012540693, 'inc'),
   (0.011404725, 'new'),
   (0.010842457, 'company'),
   (0.010076889, 'corp'),
   (0.008816432, 'fullquote'),
   (0.008758007, 'billion'),
   (0.008564316, 'percent'),
   (0.008191634, 'sales'),
   (0.0075583793, 'com'),
   (0.007341842, 'stocks'),
   (0.007296972, 'year'),
   (0.007152813, 'quarter'),
   (0.0069120284, 'target'),
   (0.006541582, 'business'),
   (0.0065185083, 'million'),
   (0.0063803825, 'ibm'),
   (0.006374945, 'gt'),
   (0.0063683908, 'buy')],
  -2.6554602194145205),
 ([(0.01225956, 'game'),
   (0.011806377, 'first'),
   (0.011282595, 'season'),
   (0.009661658, 'ap'),
   (0.008901622, 'night'),
   (0.008504875, 'team'),
   (0.008488618, 'new'),
   (0.0077206125, 'coach'),
   (0.007427938, 'last'),
   (0.0073703616, 'win'),
   (0.006860296, 'two'),
   (0.006547133, 'year'),
   (0.006322045, 'second'),
   (0.006255431, 'league'),
   (0.0062381844, 'poin

## **LSI or NMF**

In [ ]:
from gensim.models import LsiModel
from gensim.models import Nmf

### **AG News**

In [ ]:
temp = dictionary_tweets[0]  # This is only to "load" the dictionary.
id2word_ag = dictionary_tweets.id2token
# lsi_ag = LsiModel(corpus_ag, id2word=id2word_ag, num_topics = 5)

In [ ]:
temp = dictionary_ag[0]  # This is only to "load" the dictionary.
id2word_ag = dictionary_tweets.id2token
nmf_ag = Nmf(corpus_tweets, num_topics=5, id2word = id2word_ag)

In [ ]:
cm = CoherenceModel(model=nmf_ag, corpus=corpus_ag, dictionary=dictionary_ag, coherence="u_mass" , texts=preprocess_ag)
cm.get_coherence()

-14.235026881716559

In [ ]:
nmf_ag.top_topics(corpus_tweets)

[([(0.06900650328251061, 'covid'),
   (0.02601543307931705, 'https'),
   (0.020408301071713684, 'one'),
   (0.020364461725277307, 'number'),
   (0.019837958981432945, 'high'),
   (0.01980943012033739, 'patients'),
   (0.0192627828457446, 'care'),
   (0.019187507167117226, 'take'),
   (0.019065145965685478, 'days'),
   (0.018954389744560794, 'really'),
   (0.01859600095887771, 'take_care'),
   (0.018590610381037563, 'talking'),
   (0.018554954812765485, 'asking'),
   (0.018522378019930404, 'armys'),
   (0.018520367848490437, 'patients_days'),
   (0.018520367848490437, 'asking_armys'),
   (0.01851622314047905, 'really_high'),
   (0.01851517716093119, 'btstranslation_'),
   (0.01851517716093119, 'btstranslation__talking'),
   (0.007039766251467217, 'fauci')],
  -2.6372943544966483),
 ([(0.09627450403457938, 'https'),
   (0.041136012272316995, 'covid'),
   (0.016274677124375865, 'test'),
   (0.01281654655420479, 'hours'),
   (0.012777007829832267, 'woman'),
   (0.012285105942847612, 'turn'

In [ ]:
topics_ag_nmf_df = pd.DataFrame()
topics_ag_nmf = nmf_ag.top_topics(corpus_ag)
for i in range(len(topics_ag_nmf)):
  temp = []
  for j in range(14):
    # print(i[0][j][1])
    temp.append(topics_ag_nmf[i][0][j][1])
  print(temp)
  topics_ag_nmf_df[i] = temp

['china', 'test', 'hours', 'woman', 'turn', 'chicken', 'swab', 'roast', 'roast_chicken', 'swab_test', 'queue', 'queues', 'realises_queue', 'realises']
['new', 'cases', 'covid', 'last', 'covid19', 'says', 'coronavirus', 'week', 'reports', 'coronavirus_cases', 'fauci', 'bnodesk', 'reports_new', 'state']
['covid', 'number', 'one', 'high', 'patients', 'care', 'take', 'days', 'really', 'take_care', 'talking', 'asking', 'armys', 'asking_armys']
['china', 'deaths', 'first', 'breaking', 'year', 'deaths_year', 'covid19', 'drericding', 'confirmed', 'outbreaks', 'grappling', 'china_grappling', 'sustained', 'china_confirmed']
['https', 'covid', 'ivermectin', 'hospitalizations', 'largest', 'reduce', 'date', 'trial', 'hospitalizations_largest', 'trial_date', 'wsj', 'covid19', 'comments', 'peterhotez']


In [ ]:
topics_ag_nmf_df
# .to_csv("nmf_topics_tweets.csv")

### **K Selection**

In [ ]:
nmf_cm_ag = {
    "u_mass": [],
    "c_v": [], 
    "c_uci": [],
    "c_npmi": []
}

In [ ]:
for num_topic in range(2, 10, 1):
    print(f"currently run {num_topic} topic number")
    temp = dictionary_ag[0]  # This is only to "load" the dictionary.
    id2word_ag = dictionary_ag.id2token
    nmf_ag = LsiModel(corpus_ag, num_topics=num_topic, id2word = id2word_ag)

    for method in list(nmf_cm_ag.keys()):
        cm = CoherenceModel(model=nmf_ag, corpus=corpus_ag, dictionary=dictionary_ag, coherence=method , texts=preprocess_ag)
        nmf_cm_ag[method].append(cm.get_coherence())
    clear_output()

In [ ]:
pd.DataFrame(nmf_cm_ag)

,u_mass,c_v,c_uci,c_npmi
0,-2.382941,0.511827,-0.907812,0.023718
1,-3.455770,0.457573,-1.308573,0.009320
2,-4.007348,0.412112,-1.936421,-0.012414
3,-3.275263,0.515779,-0.525902,0.043660
4,-2.875275,0.533669,-0.275302,0.041654
5,-2.849893,0.525774,-0.698019,0.035259
6,-3.234072,0.506928,-0.241318,0.041935
7,-3.494662,0.484137,-0.621700,0.020161


In [ ]:
pd.DataFrame(nmf_cm_ag).to_csv("lsi_ag_cm2-9.csv")

### **Twitter COVID-19**

In [ ]:
nmf_cm_tweets = {
    "u_mass": [],
    "c_v": [], 
    "c_uci": [],
    "c_npmi": []
}

In [ ]:
for num_topic in range(2, 10, 1):
    print(f"currently run {num_topic} topic number")
    temp = dictionary_ag[0]  # This is only to "load" the dictionary.
    id2word_tweets = dictionary_tweets.id2token
    nmf_tweets = LsiModel(corpus_ag, num_topics=num_topic, id2word = id2word_tweets)

    for method in list(nmf_cm_tweets.keys()):
        cm = CoherenceModel(model=nmf_tweets, corpus=corpus_tweets, dictionary=dictionary_tweets, coherence=method , texts=preprocess_tweets)
        nmf_cm_tweets[method].append(cm.get_coherence())
    clear_output()

In [ ]:
pd.DataFrame(nmf_cm_tweets).to_csv("lsi_tweets_cm2-9.csv")

## **Topic Visualization**

In [ ]:
!pip install pyLDAvis

In [ ]:
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
pyLDAvis.enable_notebook()

/usr/local/lib/python3.7/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Iterable


In [ ]:
lda_viz = gensimvis.prepare(model_ag, corpus_ag, dictionary_ag)
clear_output()

In [ ]:
lda_viz = gensimvis.prepare(model_tweets, corpus_tweets, dictionary_tweets)
clear_output()

In [ ]:
lda_viz

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
7      0.072514  0.104396       1        1  16.980311
9     -0.101700  0.138561       2        1  12.249066
5     -0.063948 -0.071308       3        1  11.627412
0      0.023416 -0.191036       4        1  10.765250
6      0.222695  0.060276       5        1   9.895178
3      0.188513 -0.090147       6        1   9.452070
2     -0.142846  0.317970       7        1   8.432638
4      0.231070 -0.010718       8        1   7.883030
1     -0.166469 -0.103464       9        1   7.312975
8     -0.263245 -0.154531      10        1   5.402069, topic_info=         Term          Freq         Total Category  logprob  loglift
2424       gt  13366.000000  13366.000000  Default  30.0000  30.0000
2426       lt  13339.000000  13339.000000  Default  29.0000  29.0000
2564    lt_gt   9073.000000   9073.000000  Default  28.0000  28.0000
6     reuters  19627.000000  19627.000000  Default  27.0000  27.0000
128        ap  17302.000000  17302.000000  Default  26.0000  26.0000
...       ...           ...           ...      ...      ...      ...
6     reuters   5190.634344  19627.987229  Topic10  -3.5137   1.5883
89        new   2231.561964  20870.942991  Topic10  -4.3579   0.6827
115       inc   1423.199854   6572.524596  Topic10  -4.8077   1.3884
562      corp    967.908275   5259.908257  Topic10  -5.1932   1.2257
1032       co    871.549557   2804.084661  Topic10  -5.2980   1.7498

[747 rows x 6 columns], token_table=       Topic      Freq        Term
term                              
9948       8  0.998795       abbas
318        1  0.555870   according
318        2  0.125789   according
318        3  0.189774   according
318        4  0.015996   according
...      ...       ...         ...
7263       2  0.999623       yukos
15145      9  0.999685  yushchenko
5320       8  0.999112     zealand
1599       1  0.017119        zone
1599       8  0.982652        zone

[1927 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[8, 10, 6, 1, 7, 4, 3, 5, 2, 9])

## **Embedded Topic Model**

In [ ]:
!pip install embedded_topic_model

In [ ]:
from embedded_topic_model.utils import preprocessing
from embedded_topic_model.utils import embedding
from embedded_topic_model.models.etm import ETM
import json

In [ ]:
embeddings_mapping = embedding.create_word2vec_embedding_from_dataset(preprocess_ag_text)

In [ ]:

# Loading a dataset in JSON format. As said, documents must be composed by string sentences
# corpus_file = 'datasets/example_dataset.json'
# documents_raw = json.load(open(dataset, 'r'))
# documents = [document['body'] for document in documents_raw]


# Preprocessing the dataset
vocabulary, train_dataset, _, = preprocessing.create_etm_datasets(
    preprocess_ag_text, 
    min_df=0.01, 
    max_df=0.75, 
    train_size=0.85, 
)

In [ ]:
train_dataset["tokens"].shape

(101624,)

In [ ]:
etm_ag = ETM(
    vocabulary,
    embeddings=embeddings_mapping, # You can pass here the path to a word2vec file or
                                   # a KeyedVectors instance
    num_topics=5,
    epochs=100,
    # debug_mode=True,
    train_embeddings=False, # Optional. If True, ETM will learn word embeddings jointly with
                            # topic embeddings. By default, is False. If 'embeddings' argument
                            # is being passed, this argument must not be True
)
etm_ag.fit(train_dataset)

## **GSDMM**

In [ ]:
!pip install git+https://github.com/rwalk/gsdmm.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/rwalk/gsdmm.git to /tmp/pip-req-build-1jcbxbi3
  Running command git clone -q https://github.com/rwalk/gsdmm.git /tmp/pip-req-build-1jcbxbi3
  Created wheel for gsdmm: filename=gsdmm-0.1-py3-none-any.whl size=4602 sha256=b73730a331b0f40c0d405c2639049a347d92e3da73abbd2cdfa1bd817de1e622
  Stored in directory: /tmp/pip-ephem-wheel-cache-ndmnywbv/wheels/34/65/a6/7eef67b88abae954fecd22587bd755c27b58a9ffe488d6b0de
Successfully built gsdmm


In [ ]:
from gsdmm import MovieGroupProcess

In [ ]:
# K= 5, 
gsdmm_ag = MovieGroupProcess(alpha=0.1, beta=0.1, n_iters=30)

In [ ]:
gsdmm_tweets = MovieGroupProcess(n_iters = 30)

In [ ]:
vocab_length_tweets = len(dictionary_tweets)
tweets_y = gsdmm_tweets.fit(preprocess_tweets, vocab_length_tweets)

In stage 0: transferred 100613 clusters with 8 clusters populated
In stage 1: transferred 25933 clusters with 8 clusters populated
In stage 2: transferred 11201 clusters with 8 clusters populated
In stage 3: transferred 6996 clusters with 8 clusters populated
In stage 4: transferred 5024 clusters with 8 clusters populated
In stage 5: transferred 4229 clusters with 8 clusters populated
In stage 6: transferred 3723 clusters with 8 clusters populated
In stage 7: transferred 3460 clusters with 8 clusters populated
In stage 8: transferred 3331 clusters with 8 clusters populated
In stage 9: transferred 3224 clusters with 8 clusters populated
In stage 10: transferred 3113 clusters with 8 clusters populated
In stage 11: transferred 3096 clusters with 8 clusters populated
In stage 12: transferred 2935 clusters with 8 clusters populated
In stage 13: transferred 2886 clusters with 8 clusters populated
In stage 14: transferred 2807 clusters with 8 clusters populated
In stage 15: transferred 2849 c

In [ ]:
vocab_length = len(dictionary_ag)

In [ ]:
y = gsdmm_ag.fit(preprocess_ag, vocab_length)

In stage 0: transferred 100490 clusters with 8 clusters populated
In stage 1: transferred 37347 clusters with 8 clusters populated
In stage 2: transferred 10412 clusters with 8 clusters populated
In stage 3: transferred 6123 clusters with 8 clusters populated
In stage 4: transferred 4488 clusters with 8 clusters populated
In stage 5: transferred 4141 clusters with 8 clusters populated
In stage 6: transferred 3862 clusters with 8 clusters populated
In stage 7: transferred 3742 clusters with 8 clusters populated
In stage 8: transferred 3482 clusters with 8 clusters populated
In stage 9: transferred 3372 clusters with 8 clusters populated
In stage 10: transferred 3372 clusters with 8 clusters populated
In stage 11: transferred 3121 clusters with 8 clusters populated
In stage 12: transferred 3006 clusters with 8 clusters populated
In stage 13: transferred 2606 clusters with 8 clusters populated
In stage 14: transferred 2403 clusters with 8 clusters populated
In stage 15: transferred 2136 c

In [ ]:
def get_topics_lists(model, top_clusters, n_words):
    '''
    Gets lists of words in topics as a list of lists.
    
    model: gsdmm instance
    top_clusters:  numpy array containing indices of top_clusters
    n_words: top n number of words to include
    
    '''
    # create empty list to contain topics
    topics = []
    
    # iterate over top n clusters
    for cluster in top_clusters:
        #create sorted dictionary of word distributions
        sorted_dict = sorted(model.cluster_word_distribution[cluster].items(), key=lambda k: k[1], reverse=True)[:n_words]
         
        #create empty list to contain words
        topic = []
        
        #iterate over top n words in topic
        for k,v in sorted_dict:
            #append words to topic list
            topic.append(k)
            
        #append topics to topics list    
        topics.append(topic)
    
    return topics

In [ ]:
def top_words(cluster_word_distribution, top_cluster, values):
    for cluster in top_cluster:
        sort_dicts =sorted(gsdmm_ag.cluster_word_distribution[cluster].items(), key=lambda k: k[1], reverse=True)[:values]
        print('Cluster %s : %s'%(cluster,sort_dicts))
        print('-'*120)

In [ ]:
np.array(gsdmm_ag.cluster_doc_count)

array([22046, 15558, 21418, 12162,  8042, 13757, 12803, 14214])

In [ ]:
doc_count = np.array(gsdmm_ag.cluster_doc_count)
top_index = doc_count.argsort()[-15:][::-1]
topics = get_topics_lists(gsdmm_ag, top_index, 10) 

In [ ]:
temp_topics = pd.DataFrame()
for i in range(len(topics)):
  temp_topics[i] = topics[i]

In [ ]:
temp_topics.to_csv("gsdmm_topics_ag.csv")

In [ ]:
doc_count_tweets = np.array(gsdmm_tweets.cluster_doc_count)
top_index_tweets = doc_count_tweets.argsort()[-15:][::-1]
topics_tweets = get_topics_lists(gsdmm_tweets, top_index_tweets, 10) 

In [ ]:
temp_topics_tweets = pd.DataFrame()
for i in range(len(topics_tweets)):
  temp_topics_tweets[i] = topics_tweets[i]

In [ ]:
temp_topics_tweets.to_csv("gsdmm_topics_tweets.csv")

In [ ]:
# get topics to feed to coherence model
doc_count = np.array(gsdmm_ag.cluster_doc_count)
top_index = doc_count.argsort()[-15:][::-1]
topics = get_topics_lists(gsdmm_ag, top_index, 20) 

# evaluate model using Topic Coherence score
cm_gsdmm = CoherenceModel(topics=topics, 
                          dictionary=dictionary_ag, 
                          corpus=corpus_ag, 
                          texts=preprocess_ag, 
                          coherence='u_mass')

# get coherence value
coherence_gsdmm = cm_gsdmm.get_coherence()  

print(coherence_gsdmm)

-3.0778875529652554


In [ ]:
top_words(gsdmm_ag.cluster_word_distribution, top_index, 10)

Cluster 0 : [('first', 3088), ('new', 2805), ('game', 2800), ('season', 2670), ('win', 2648), ('team', 2602), ('night', 2261), ('two', 2231), ('coach', 2228), ('one', 2178)]
------------------------------------------------------------------------------------------------------------------------
Cluster 2 : [('new', 5780), ('microsoft', 5294), ('software', 3510), ('internet', 3191), ('company', 2581), ('search', 2400), ('music', 2354), ('service', 2320), ('said', 2294), ('quot', 2112)]
------------------------------------------------------------------------------------------------------------------------
Cluster 1 : [('said', 4726), ('iraq', 4394), ('reuters', 3047), ('killed', 2903), ('two', 2553), ('iraqi', 2319), ('baghdad', 2239), ('minister', 2134), ('people', 2001), ('palestinian', 1933)]
------------------------------------------------------------------------------------------------------------------------
Cluster 7 : [('said', 3893), ('company', 3346), ('new', 2477), ('inc', 2452

### **Find Optimal K**

In [ ]:
gsdmm_coherence_ag = {
    "u_mass": [],
    "c_v": [], 
    "c_uci": [],
    "c_npmi": []
}

In [ ]:
for num_topic in range(2, 10, 1):
    print(f"currently run {num_topic} topic number")
    vocab_length = len(dictionary_ag)
    gsdmm_ag = MovieGroupProcess(K= num_topic, alpha=0.1, beta=0.1, n_iters=10)

    # evaluate model using Topic Coherence score
    gsdmm_ag.fit(preprocess_ag, vocab_length)

    doc_count = np.array(gsdmm_ag.cluster_doc_count)
    top_index = doc_count.argsort()[-15:][::-1]
    topics = get_topics_lists(gsdmm_ag, top_index, 20) 

    for method in list(gsdmm_coherence_ag):
        cm_gsdmm = CoherenceModel(topics=topics, 
                              dictionary=dictionary_ag, 
                              corpus=corpus_ag, 
                              texts=preprocess_ag, 
                              coherence=method)
        gsdmm_coherence_ag[method].append(cm_gsdmm.get_coherence())
    clear_output()

In [ ]:
pd.DataFrame(gsdmm_coherence_ag).to_csv("gsdmm_ag_cm.csv")

#### **Twitter Dataset**

In [ ]:
gsdmm_coherence_tweets = {
    "u_mass": [],
    "c_v": [], 
    "c_uci": [],
    "c_npmi": []
}

In [ ]:
dictionary_tweets[3]

'continued'

In [ ]:
for num_topic in range(2, 10, 1):
    print(f"currently run {num_topic} topic number")
    vocab_length = len(dictionary_tweets)
    gsdmm_tweet = MovieGroupProcess(K= num_topic, alpha=0.1, beta=0.1, n_iters=10)

    # evaluate model using Topic Coherence score
    gsdmm_tweet.fit(preprocess_tweets, vocab_length)

    doc_count = np.array(gsdmm_tweet.cluster_doc_count)
    top_index = doc_count.argsort()[-15:][::-1]
    topics = get_topics_lists(gsdmm_tweet, top_index, 20) 

    for method in list(gsdmm_coherence_tweets):
        cm_gsdmm = CoherenceModel(topics=topics, 
                              dictionary=dictionary_tweets, 
                              corpus=corpus_tweets, 
                              texts=preprocess_tweets, 
                              coherence=method)
        gsdmm_coherence_tweets[method].append(cm_gsdmm.get_coherence())
    clear_output()

In [ ]:
pd.DataFrame(gsdmm_coherence_tweets).to_csv("gsdmm_tweets_cm2-9.csv")

## **BTM**

In [ ]:
!pip install biterm

In [ ]:
from biterm.utility import vec_to_biterms
from biterm.btm import oBTM

In [ ]:
# X = countvec_ag.toarray()

In [ ]:
vocab = np.array(vectorizer.get_feature_names())

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
biterms = vec_to_biterms(corpus_ag)

In [ ]:
btm = oBTM(num_topics=5, V=vocab)
topics = btm.fit_transform(biterms, iterations=100)

## **BERTopic**

In [ ]:
!pip install bertopic

In [ ]:
from bertopic import BERTopic

In [ ]:
topic_model = BERTopic(embedding_model = "all-MiniLM-L6-v2", nr_topics = 2)
topics, probs = topic_model.fit_transform(preprocess_ag_text)

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/10.2k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/612 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/350 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/numba/np/ufunc/parallel.py:363: NumbaWarning: The TBB threading layer requires TBB version 2019.5 or later i.e., TBB_INTERFACE_VERSION >= 11005. Found TBB_INTERFACE_VERSION = 9107. The TBB threading layer is disabled.
  warnings.warn(problem)


In [ ]:
topic_model.get_topic_info()

,Topic,Count,Name
0,-1,33731,-1_baghdad_iran_profit_nuclear
1,0,1318,0_stocks_investors_higher_wall
2,1,1027,1_crude_barrel_heating_winter
3,2,735,2_search_engine_desktop_google
4,3,709,3_kerry_john_bush_democratic
...,...,...,...
2049,2071,10,2071_coral_reefs_reef_oceans
2050,2073,10,2073_saudi_riyadh_dissident_reformists
2051,2074,10,2074_nordstrom_estee_lauder_jwn
2052,2075,10,2075_dissident_detained_detains_intellectuals


In [ ]:
topic_model.save("ag_news_bert")

In [ ]:
topic_model.get_topics()

In [ ]:
topic_model_tweets = BERTopic(embedding_model = "all-MiniLM-L6-v2")
topics_tweets, probs_tweets = topic_model_tweets.fit_transform(preprocess_tweets_text)

In [ ]:
topic_model_tweets.get_topics()

In [ ]:
topic_model_tweets.save("tweets_bert")

/usr/local/lib/python3.7/dist-packages/scipy/sparse/_index.py:126: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


### **BERT Coherence**

In [ ]:
topic_model.get_topics()

TypeError: ignored

In [ ]:
for i in topic_model.get_topic(i):
  print(i[0])

nikkei
tokyo
average
exporters
yen
midday
midsession
japanese
stocks
gains


In [ ]:
# AG News top 10 topic for coherence
topic_bert_ag = []
for i in range(-1, 9):
  temp = []
  for j in topic_model.get_topic(i):
    temp.append(j[0])
  topic_bert_ag.append(temp)

In [ ]:
# Twitter COVID-19 top 10 topic for coherence
topic_bert_tweets = []
for i in range(-1, 9):
  temp = []
  for j in topic_model_tweets.get_topic(i):
    temp.append(j[0])
  topic_bert_tweets.append(temp)

In [ ]:
topic_model_tweets.get_topics()

{-1: [('https', 0.0008288188078527397),
  ('thanks', 0.0007580981100519382),
  ('long', 0.0007308554891899524),
  ('sure', 0.0006884748175857826),
  ('want', 0.0006831870927671619),
  ('countries', 0.0006735771245166293),
  ('many', 0.000670542411768906),
  ('like', 0.0006670169591601272),
  ('think', 0.0006645030526906961),
  ('good', 0.0006642833971970202)],
 0: [('btstranslation_', 0.009114061352267186),
  ('armys', 0.009109713997678928),
  ('asking', 0.008993495050681899),
  ('talking', 0.008929235951722687),
  ('care', 0.007971453591888968),
  ('really', 0.007949562378462104),
  ('take', 0.007697031505813158),
  ('days', 0.007638243742764378),
  ('patients', 0.007494621952988677),
  ('high', 0.007420812484565612)],
 1: [('drsimonegold', 0.012838762851736187),
  ('institute', 0.01251919863889133),
  ('additional', 0.012513344442660843),
  ('necessary', 0.012508660725333257),
  ('stop', 0.010363994486904334),
  ('restrictions', 0.010262253568412651),
  ('may', 0.0100669920766075),
 

In [ ]:
bert_ag_cm = {}

In [ ]:
for i in ['u_mass', 'c_v', 'c_uci', 'c_npmi']:
  cm_bert = CoherenceModel(topics=topic_bert_ag, 
                        dictionary=dictionary_ag, 
                        corpus=corpus_ag, 
                        texts=preprocess_ag, 
                        coherence=i)
  bert_ag_cm[i] = cm_bert.get_coherence()
  print(f"{i} coherence: {bert_ag_cm[i]}")

u_mass coherence: -3.451866041602396
c_v coherence: 0.8189382750419183
c_uci coherence: 1.163475173995769
c_npmi coherence: 0.21750537818774748


In [ ]:
bert_tweets_cm = {}

In [ ]:
for i in ['u_mass', 'c_v', 'c_uci', 'c_npmi']:
  cm_bert = CoherenceModel(topics=topic_bert_tweets, 
                        dictionary=dictionary_tweets, 
                        corpus=corpus_tweets, 
                        texts=preprocess_tweets, 
                        coherence=i)
  bert_tweets_cm[i] = cm_bert.get_coherence()
  print(f"{i} coherence: {bert_tweets_cm[i]}")

u_mass coherence: -1.2723701893642183
c_v coherence: 0.8923964685160977
c_uci coherence: 1.669053233887844
c_npmi coherence: 0.5206940923907758


In [ ]:
!pip install lda2vec

# **Evaluation**

## **Coherence Plot for K Selection**

In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [ ]:
cm_lda_ag = pd.read_csv("lda_agnews_cm2-9.csv", index_col = 0)
cm_lda_tweets = pd.read_csv("lda_tweets_cm2-9.csv", index_col = 0)
cm_gsdmm_ag = pd.read_csv("gsdmm_ag_cm2-9.csv", index_col = 0)
cm_gsdmm_tweets = pd.read_csv("gsdmm_tweets_cm2-9.csv", index_col = 0)
# cm_lsi_ag = pd.read_csv("lsi_ag_cm2-9.csv", index_col = 0)
# cm_lsi_tweets = pd.read_csv("lsi_tweets_cm2-9.csv", index_col = 0)
cm_nmf_ag = pd.read_csv("nmf_ag_cm2-9.csv", index_col = 0)
cm_nmf_tweets = pd.read_csv("nmf_tweets_cm2-9.csv", index_col = 0)

In [ ]:
def compare_coherence(df, model, dataset):
  df["k"] = [i for i in range(2, 10)]
  fig = make_subplots(rows=2, cols=2, subplot_titles=("u_mass coherence", "c_v coherence", \
                                                    "c_uci coherence", "c_npmi coherence"),
                      x_title='Topic Number',
                      y_title='Coherence Score',)
  
  fig.add_trace(go.Scatter(x=df["k"], y=df["u_mass"],
                      mode='lines+markers',
                      name='u_mass'), row = 1, col = 1)
  
  fig.add_trace(go.Scatter(x=df["k"], y=df["c_v"],
                      mode='lines+markers',
                      name='c_v'), row = 1, col = 2)
  
  fig.add_trace(go.Scatter(x=df["k"], y=df["c_uci"],
                      mode='lines+markers',
                      name='c_uci'), row = 2, col = 1)
  
  fig.add_trace(go.Scatter(x=df["k"], y=df["c_npmi"],
                      mode='lines+markers',
                      name='c_npmi'), row = 2, col = 2)
  fig.update_layout(height=500, width=700,
                    title_text=f"{model} Topic Coherence Comparison with {dataset} Dataset")
  

  fig.show()

In [ ]:
compare_coherence(cm_lda_ag, "LDA", "AG News")

In [ ]:
compare_coherence(cm_lda_tweets, "LDA", "Tweets")

In [ ]:
compare_coherence(cm_gsdmm_ag, "GSDMM", "AG News")

In [ ]:
compare_coherence(cm_gsdmm_tweets, "GSDMM", "Tweets")

In [ ]:
compare_coherence(cm_nmf_ag, "NMF", "AG News")

In [ ]:
compare_coherence(cm_nmf_tweets, "NMF", "Tweets")

## **Coherence Model Comparison**

In [ ]:
compare_list_ag = [cm_lda_ag, cm_gsdmm_ag, cm_nmf_ag]
compare_list_tweets = [cm_lda_tweets, cm_gsdmm_tweets, cm_nmf_tweets]
model_list = ["LDA", "GSDMM", "NMF"]

In [ ]:
def compare_model(df_list, cm, dataset):
  for i in df_list:
    i["k"] = [j for j in range(2, 10)]
  model_list = ["LDA", "GSDMM", "NMF"]
  # fig = make_subplots(rows=2, cols=2, subplot_titles=("u_mass coherence", "c_v coherence", \
  #                                                   "c_uci coherence", "c_npmi coherence"))
  fig = go.Figure()
  for i in range(len(df_list)):
    fig.add_trace(go.Scatter(x=df_list[i]["k"], y=df_list[i][cm],
                        mode='lines+markers',
                        name=model_list[i]))
  # fig.add_trace(px.line(df, x='k', y='u_mass', color='model'), row = 1, col = 1)
  
  
  # for i in range(len(df_list)):
  #   fig.add_trace(go.Scatter(x=df_list[i]["k"], y=df_list[i]["c_v"],
  #                       mode='lines+markers',
  #                       name=model_list[i]), row = 1, col = 2)
  
  # for i in range(len(df_list)):
  #   fig.add_trace(go.Scatter(x=df_list[i]["k"], y=df_list[i]["c_uci"],
  #                       mode='lines+markers',
  #                       name=model_list[i]), row = 2, col = 1)
  
  # for i in range(len(df_list)):
  #   fig.add_trace(go.Scatter(x=df_list[i]["k"], y=df_list[i]["c_npmi"],
  #                       mode='lines+markers',
  #                       name=model_list[i]), row = 2, col = 2)
  fig.update_layout(height=500, width=700,
                    title_text=f"{cm} Topic Coherence Comparison with {dataset} Dataset")
  fig.show()

In [ ]:
cm_list = ["u_mass", "c_v", "c_uci", "c_npmi"]

In [ ]:
for i in cm_list:
  compare_model(compare_list_ag, i, "AG News")

In [ ]:
for i in cm_list:
  compare_model(compare_list_tweets, i, "Tweets")